In [11]:
import nltk 
import glob, os
import re
import string
import sys
import random
import xml.etree.ElementTree as ET
import xml.etree
import numpy

from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from collections import Counter
from nltk.corpus import stopwords
import html
from lxml import etree
import unidecode



In [12]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/chryssa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/chryssa/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/chryssa/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chryssa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
def checkQuality(sentence):
    
    lemmatizer = WordNetLemmatizer() 
    tokenized_text = nltk.word_tokenize(sentence)
    length = len(tokenized_text)
    alpha = 0
    nonAlpha = 0
    found = 0
    nonFound = 0
    upper = 0
    stop = 0
    stop_words = set(stopwords.words('english')) 
    
    for token in tokenized_text:
        #print(token)
        if len(token)==1:
            if token.isalpha():
                alpha+=1
            else:
                nonAlpha+=1
        else:
            if(token.lower() in stop_words):
                stop+=1
            else:
                lemma = lemmatizer.lemmatize(token.lower())
                if len(wn.synsets(lemma))>0:
                    found+=1
                else:
                    if token.isalpha():
                        if not token.islower() and not token.isupper():
                            upper+=1
                            #print(token)
                        elif token.isupper():
                            upper+=1
                        else:
                            nonFound+=1
                    else:
                        nonAlpha+=1
    alphaS = alpha/length
    nonAlphaS = nonAlpha/length
    foundS = found/length
    nonFoundS = nonFound/length
    upperS = upper/length
    stopS = stop/length
    nv = False
    if(nonFoundS>0.3):
        #print(sentence + "====>NF:" + str(nonFound))
        nv = True
    if(foundS<0.11):
        #print(sentence + "====>F:" + str(foundS))
        nv = True
    if(alphaS>0.2):
        #print(sentence + "====>AF:" + str(alphaS))
        nv = True
    if nv:
        return False
    return True


In [12]:
def filtering(senText, buffSent):
    mat = False
    p = re.compile('\([a-zA-Z]\)')
    #print(senText)
    re.sub(' +', ' ', senText)
    
    m = p.match(senText)

    if m:
        if m.start() == 0:
            mat = True
    if senText.strip().startswith("(") and not mat:
        if buffSent.endswith("."):
            return True
    elif (buffSent.strip().endswith(". . .") or buffSent.strip().endswith("...")):
        return True
    elif not senText[0].isupper() and not mat:
        if buffSent.strip().endswith(")"):
            return True


In [14]:
def parseSentXMLtree(xmlTree, offsets):
    buffSent = ""
    buffSid = []
    buffSsid = []
    entries = []

    for sent in xmlTree.iter("S"):
        mat = False
        fixed = False
        sid = sent.attrib['sid']
        if sid == "0" or sid =="":
            ssid = "0"
        else:
            ssid = sent.attrib['ssid']
        senText = sent.text
       
        if(len(offsets)==1):
            entries.append([senText, sid, ssid])

        else:
            #fixing
            if senText!=None:

                fixed = filtering(senText, buffSent)
                if (not fixed) and (len(buffSid)>0):
                    entries.append([buffSent, str(buffSid), str(buffSsid)]) 
                    buffSid.clear()
                    buffSsid.clear()
                    buffSent = ""
                
                buffSid.append(sid)
                buffSsid.append(ssid)
                buffSent+=senText

    if len(offsets)>1:
        entries.append([buffSent, str(buffSid), str(buffSsid)])  

    buffSid.clear()
    buffSsid.clear()
    buffSent = ""
    
    return entries

In [17]:
### parse reference sentence xmls to be used for generation of negative pairs ###

def parse_references(year='2019',docType='Reference'):

    filenames =[]
    fileCounter = 0
    p = re.compile('\([a-zA-Z]\)')
    initDir = "CL-SciSumm/scisumm-corpus-master/data/Training-Set-"+year+"/"
    writeDir = "CL-SciSumm/scisumm-corpus-master/data/"+docType +"_Sent-"+year+"/"
    writeDir2 = "CL-SciSumm/scisumm-corpus-master/data/"+docType +"_Sent-"+year+"_plain/"

    if not os.path.exists(writeDir):
        os.makedirs(writeDir)
    if not os.path.exists(writeDir2):
        os.makedirs(writeDir2)

    for root, dirs, files in os.walk(initDir):
        dir = os.path.basename(root)
        if dir == docType + "_XML":
            for file in files:
                if file.endswith(".xml"):
                    fileStr=str(root)
                    fileStr+="/"
                    fileStr+=file
                    #print(fileStr)
                    if not file in filenames:
                        filenames.append(file)
                        with open(fileStr, encoding='utf-8', errors='ignore') as xml2str:
                            data=xml2str.read()
                            parser = ET.XMLParser()
                            xmlTree = ET.fromstring(data,parser)
                            buffSent = ""
                            buffSid = []
                            buffSsid = []


                            wf = open(writeDir+file.split(".")[0]+"_sentences.txt", "w") 
                            wf2 = open(writeDir2+file.split(".")[0]+"_sentences_plain.txt", "w") 
                            fileCounter+=1
                            for sent in xmlTree.iter("S"):
                                mat = False
                                fixed = False
                                sid = sent.attrib['sid']
                                if sid == "0" or sid =="":
                                    ssid = "0"
                                else:
                                    ssid = sent.attrib['ssid']
                                senText = sent.text

                                #fix parsing
                                if senText!=None:
                                    fixed = filtering(senText,buffSent)
                                else:
                                    fixed = False
                                if (not fixed) and (len(buffSid)>0):
                                    wf.write(buffSent+"\t"+str(buffSid)+"\t"+str(buffSsid)+"\n") 
                                    wf2.write(buffSent+"\n") 
                                    buffSid.clear()
                                    buffSsid.clear()
                                    buffSent = ""
                                if senText!=None:   
                                    buffSid.append(sid)
                                    buffSsid.append(ssid)
                                    buffSent+=senText

                            wf.close()
    print(fileCounter)

In [ ]:
parse_references('2019','Reference')
parse_references('2018','Reference')
parse_references('2018','Citance')

In [18]:
#generate positive pairs 
def generate_positive_pairs(year):


    initDir = "CL-SciSumm/scisumm-corpus-master/From-ScisummNet-"+year+"/"
    annotationFile = ".ann"
    counter = 0;
    paircount = 0;
    writeDir1 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_offsets_pos-"+year+"/"
    writeDir2 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_pos-"+year+"/"
    writeDir3 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_offsets_full_pos-"+year+"/"
    writeDir4 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_full_pos-"+year+"/"

    if not os.path.exists(writeDir1):
        os.makedirs(writeDir1)
    if not os.path.exists(writeDir2):
        os.makedirs(writeDir2)
    if not os.path.exists(writeDir3):
        os.makedirs(writeDir3)
    if not os.path.exists(writeDir4):
        os.makedirs(writeDir4)

    for root, dirs, files in os.walk(initDir):
        for file in files:    
             if annotationFile in file:
                with open (os.path.join(root, file), "r") as annFile:
                    counter+=1
                    wf1 = open(writeDir1+file.split(".")[0]+"_pairs_off.tsv", "w") 
                    wf2 = open(writeDir2+file.split(".")[0]+"_pairs.tsv", "w")    
                    wf3 = open(writeDir3+file.split(".")[0]+"_pairs_off.tsv", "w") 
                    wf4 = open(writeDir4+file.split(".")[0]+"_pairs.tsv", "w")    
                    for line in annFile:
                        if(len(line)>1):
                            xmlCitText = ""
                            xmlRefText = ""

                            cit_num = "0"
                            fields  = line.split(" | ")
                            for field in fields:
                                if field.startswith("Citance Number:"):
                                    start = field.find(":")
                                    cit_num = field[start+1:].strip()
                                elif(field.startswith("Reference Article:")):
                                    start = field.find(":")
                                    if("." in field):
                                        end = field.find(".", start)
                                        name_R = field[start+1:end].strip()
                                    else:
                                         name_R = field[start+1:].strip()
                                elif field.startswith("Citing Article:"):
                                    start = field.find(":") 
                                    if("." in field):
                                        end = field.find(".", start)
                                        name_C = field[start+1:end].strip()
                                    else:
                                         name_C = field[start+1:].strip()
                                elif field.startswith("Citation Offset"):
                                    start = field.find("\[") 
                                    end = field.find("\]", start)
                                    citOffset = field[start+1:end]
                                    citOffsets = citOffset.split(",")
                                elif field.startswith("Citation Text:"):
                                    start = field.find(":") 
                                    end = field.find(">", start)
                                    xmlCitText = field[start+1:].strip()
                                elif field.startswith("Reference Offset"):
                                    start = field.find("\[") 
                                    end = field.find("\]", start)
                                    refOffset = field[start+1:end]
                                    refOffsets = refOffset.split(",")
                                elif field.startswith("Reference Text:"):
                                    start = field.find(":") 
                                    xmlRefText = field[start+1:].strip()

                            parser1 = etree.XMLParser(recover=True)
                            parser2 = etree.XMLParser(recover=True)

                            try:
                                xmlEntryCit = "<root>"+xmlCitText.replace(" sid = "," sid =\"0\"")+"</root>"

                                xmlCitTree = ET.fromstring(xmlEntryCit.encode('UTF-8', 'ignore'),parser1)
                            except:
                                print("Except:"+xmlEntryCit)
                                print("\n"+str(counter))
                                raise

                            try:
                                xmlEntryRef="<root> "+xmlRefText.replace(" sid = "," sid =\"").replace(" ssid =","\" ssid =").replace("ssid = ","ssid = \"0\"").replace("&", "\&")+" </root>"

                                xmlRefTree = ET.fromstring(xmlEntryRef.encode('utf-8', 'ignore'),parser2)
                            except:
                                print("Except2:"+xmlEntryRef)
                                print("\n"+str(counter))
                                raise


                            citEntries = parseSentXMLtree(xmlCitTree, citOffsets)

                            refEntries = parseSentXMLtree(xmlRefTree, refOffsets)

                            for citEntry in citEntries:

                                if not str(citEntry[0])=="None":
                                    for refEntry in refEntries:

                                        pair_off = citEntry[0] + "\t" + refEntry[0] + "\t" +  "\t" +  "\t" + str(refEntry[1]) + "\t" + "\t" + cit_num + "\t" + name_C
                                        pair = citEntry[0] + "\t" + refEntry[0]
                                        paircount = paircount+1
                                        wf1.write(pair_off + "\n")
                                        wf2.write(pair + "\n")


                            all_cit = ""
                            all_sid = []
                            all_ssid = []
                            if len(citEntries)>1:
                                print(citEntries)
                                for citEntry in citEntries:
                                    all_cit+=citEntry[0]+" "
                                    all_sid.append(citEntry[1])
                                    all_ssid.append(citEntry[1])
                                all_cit = all_cit.strip()
                                for refEntry in refEntries:
                                    pair_off = all_cit + "\t" + refEntry[0] + "\t" +  "\t" + "\t" + str(refEntry[1]) + "\t" + cit_num + "\t" + name_C
                                    pair = all_cit + "\t" + refEntry[0] 
                                    wf3.write(pair_off + "\n")
                                    wf4.write(pair + "\n")   
                            citEntries.clear()
                            refEntries.clear()
    print(paircount)         
    print(counter)


In [ ]:
generate_positive_pairs('2018')
generate_positive_pairs('2019')

In [26]:
#process negative pairs for CL-SciSumm manually annotated data (2018)
#40 docs
#generate all possible pairs (exhaustive)


year = "2018"
initDir = "CL-SciSumm/scisumm-corpus-master/data/Training-Set-"+year+"/"
annotationFile = ".ann"


counter = 0;
paircount = 0;

writeDir1 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_offsets_corr_exhaustive_neg-"+year+"/"
writeDir2 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_corr_exhaustive_neg-"+year+"/"
writeDir3 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_offsets_full_corr_exhaustive_neg-"+year+"/"
writeDir4 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_full_corr_exhaustive_neg-"+year+"/"

if not os.path.exists(writeDir1):
    os.makedirs(writeDir1)
if not os.path.exists(writeDir2):
    os.makedirs(writeDir2)
if not os.path.exists(writeDir3):
    os.makedirs(writeDir3)
if not os.path.exists(writeDir4):
    os.makedirs(writeDir4)
for root, dirs, files in os.walk(initDir):
    for file in files:    
        if annotationFile in file:
             with open (os.path.join(root, file), "r") as annFile:
                counter+=1
               
                wf1 = open(writeDir1+file.split(".")[0]+"_pairs_off.tsv", "w") 
                wf2 = open(writeDir2+file.split(".")[0]+"_pairs.tsv", "w")    
                wf3 = open(writeDir3+file.split(".")[0]+"_pairs_off.tsv", "w") 
                wf4 = open(writeDir4+file.split(".")[0]+"_pairs.tsv", "w")    
                
                for line in annFile:
                    if(len(line)>1):
                        xmlCitText = ""
                        xmlRefText = ""
                        
                        cit_num = "0"
                        fields  = line.split(" | ")
                        for field in fields:
                            if field.startswith("Citance Number:"):
                                start = field.find(":")
                                cit_num = field[start:].strip()
                            elif(field.startswith("Reference Article:")):
                                start = field.find(":")
                                if("." in field):
                                    end = field.find(".", start)
                                    name_R = field[start+1:end].strip()
                                else:
                                     name_R = field[start+1:].strip()
                            elif field.startswith("Citing Article:"):
                                start = field.find(":") 
                                if("." in field):
                                    end = field.find(".", start)
                                    name_C = field[start+1:end].strip()
                                else:
                                     name_C = field[start+1:].strip()
                            elif field.startswith("Citation Offset"):
                                start = field.find("\[") 
                                end = field.find("\]", start)
                                citOffset = field[start+1:end]
                                citOffsets = citOffset.split(",")
                            elif field.startswith("Citation Text:"):
                                start = field.find(":") 
                                end = field.find(">", start)
                                xmlCitText = field[start+1:].strip()
                            elif field.startswith("Reference Offset"):
                                start = field.find("\[") 
                                end = field.find("\]", start)
                                refOffset = field[start+1:end]
                                refOffsets = refOffset.split(",")
                            elif field.startswith("Reference Text:"):
                                start = field.find(":") 
                                xmlRefText = field[start+1:].strip()
                        
                        #print(name_C)
                        
                        refer_sentences_file = "CL-SciSumm/scisumm-corpus-master/data/Reference_Sent-"+year+"/"+name_R+"_sentences.txt"
                   
                        lines = []
                        negSentences= []
                        if(os.path.isfile(refer_sentences_file)):
                            with open(refer_sentences_file) as f:
                                lines = f.read().splitlines()
                            max = len(lines)-1
                            min = 0
                            neg = [];
                            
                            for lineNum in range(1,len(lines)):
                                
                                candidate = lines[lineNum]
                                #print(candidate.split("\t")[1])
                                offsets = candidate.split("\t")[1].strip("\[\]").split(",")
                                valid = True
                                if not lineNum in neg:
                                    for offset in offsets:
                                        if offset in refOffsets:
                                             valid = False
                                    qual = False
                                    if valid:
                                        qual = checkQuality(candidate.split("\t")[0])
                                    if valid and qual:
                                        neg.append(lineNum)
                                        negSentences.append(candidate)
                                        
                        else:
                            print("error")
                            print(refer_sentences_file)
                            print(file)    
                        
                        parser1 = etree.XMLParser(recover=True)
                       
                        #print("Test:"+xmlCitText)
                        try:
                            xmlEntryCit = "<root>"+xmlCitText+"</root>"
                          
                            xmlCitTree = ET.fromstring(xmlEntryCit.encode('UTF-8', 'ignore'),parser1)
                        except:
                            print("Except:"+xmlEntryCit)
                            print("\n"+str(counter))
                            raise
                        
                        citEntries = parseSentXMLtree(xmlCitTree, citOffsets)
                                                
                        for citEntry in citEntries:
                            if(citEntry[0]!=None):
                                for negSent in negSentences:
                                    negSentEnt = negSent.split("\t")
                                    pair_off = citEntry[0] + "\t" + negSentEnt[0] + "\t" + str(citEntry[1]) + "\t" + str(citEntry[2]) + "\t" + str(negSentEnt[1]) + "\t" + str(negSentEnt[2]) + "\t" + name_C + "\t" 
                                    pair = citEntry[0] + "\t" + negSentEnt[0] 
                                    paircount = paircount+1
                                    wf1.write(pair_off + "\n")
                                    wf2.write(pair + "\n")
                        
                        
                        all_cit = ""
                        all_sid = []
                        all_ssid = []
                        for citEntry in citEntries:
                            if(citEntry[0]!=None):
                                all_cit+=citEntry[0]+" "
                                all_sid.append(citEntry[1])
                                all_ssid.append(citEntry[1])
                        all_cit = all_cit.strip()
                        for negSent in negSentences:
                            negSentEnt = negSent.split("\t")
                            pair_off = all_cit + "\t" + negSentEnt[0] + "\t" + str(all_sid) + "\t" + str(all_ssid) + "\t" + str(negSentEnt[1]) + "\t" + str(negSentEnt[2]) + "\t" + name_C + "\t" 
                            pair = all_cit + "\t" + negSentEnt[0] 
                            wf3.write(pair_off + "\n")
                            wf4.write(pair + "\n")
                        citEntries.clear()
                        negSentences.clear()

print(paircount)         
print(counter)

196228
40


In [ ]:
#process negative pairs for ScisummNet data (2019)
#1000 docs
#generate all possible pairs (exhaustive)

year = "2019"
initDir = "/home/chryssa/Work/CL-SciSumm/scisumm-corpus-master/From-ScisummNet-"+year+"_N/"
annotationFile = ".ann"
p = re.compile('\([a-zA-Z]\)')
re.sub(' +', ' ', senText)
m = p.match(senText)

counter = 0;
paircount = 0;
writeDir1 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_offsets_corr_exhaustive_neg-"+year+"/"
writeDir2 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_corr_exhaustive_neg-"+year+"/"
writeDir3 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_offsets_full_corr_exhaustive_neg-"+year+"/"
writeDir4 = "CL-SciSumm/scisumm-corpus-master/data/Training_pairs_full_corr_exhaustive_neg-"+year+"/"

if not os.path.exists(writeDir1):
    os.makedirs(writeDir1)
if not os.path.exists(writeDir2):
    os.makedirs(writeDir2)
if not os.path.exists(writeDir3):
    os.makedirs(writeDir3)
if not os.path.exists(writeDir4):
    os.makedirs(writeDir4)
    
for root, dirs, files in os.walk(initDir):
    for file in files:    
        if annotationFile in file:
             with open (os.path.join(root, file), "r") as annFile:
                counter+=1
                wf1 = open(writeDir1+file.split(".")[0]+"_pairs_off.tsv", "w") 
                wf2 = open(writeDir2+file.split(".")[0]+"_pairs.tsv", "w")    
                wf3 = open(writeDir3+file.split(".")[0]+"_pairs_off.tsv", "w") 
                wf4 = open(writeDir4+file.split(".")[0]+"_pairs.tsv", "w")    
        
                
                for line in annFile:
                    if(len(line)>1):
                        xmlCitText = ""
                        xmlRefText = ""
                        
                        cit_num = "0"
                        fields  = line.split(" | ")
                        for field in fields:
                            if field.startswith("Citance Number:"):
                                start = field.find(":")
                                cit_num = field[start:].strip()
                            elif(field.startswith("Reference Article:")):
                                start = field.find(":")
                                if("." in field):
                                    end = field.find(".", start)
                                    name_R = field[start+1:end].strip()
                                else:
                                     name_R = field[start+1:].strip()
                            elif field.startswith("Citing Article:"):
                                start = field.find(":") 
                                if("." in field):
                                    end = field.find(".", start)
                                    name_C = field[start+1:end].strip()
                                else:
                                     name_C = field[start+1:].strip()
                            elif field.startswith("Citation Offset"):
                                start = field.find("\[") 
                                end = field.find("\]", start)
                                citOffset = field[start+1:end]
                                citOffsets = citOffset.split(",")
                            elif field.startswith("Citation Text:"):
                                start = field.find(":") 
                                end = field.find(">", start)
                                xmlCitText = field[start+1:].strip()
                            elif field.startswith("Reference Offset"):
                                start = field.find("\[") 
                                end = field.find("\]", start)
                                refOffset = field[start+1:end]
                                refOffsets = refOffset.split(",")
                            elif field.startswith("Reference Text:"):
                                start = field.find(":") 
                                xmlRefText = field[start+1:].strip()
                        
                        #print(name_C)
                        
                        refer_sentences_file = "CL-SciSumm/scisumm-corpus-master/data/Reference_Sent-"+year+"/"+name_R+"_sentences.txt"
                   
                        lines = []
                        negSentences= []
                        if(os.path.isfile(refer_sentences_file)):
                            with open(refer_sentences_file) as f:
                                lines = f.read().splitlines()
                                print(refer_sentences_file)
                            max = len(lines)-1
                            min = 0
                            neg = [];
                            if max>0:
                                for lineNum in range(1,len(lines)):
                                    candidate = lines[lineNum]
                                    #print(candidate.split("\t")[1])
                                    #print(candidate.split("\t")[1])
                                    valid = True
                                    if len(candidate.split("\t"))>1:
                                        offsets = candidate.split("\t")[1].strip("\[\]").split(",")
                                    else:
                                        valid = False
                                    if not lineNum in neg:
                                        for offset in offsets:
                                            if offset in refOffsets:
                                                 valid = False
                                        qual = False
                                        if valid:
                                            qual = checkQuality(candidate.split("\t")[0])
                                        if valid and qual:
                                            neg.append(lineNum)
                                            negSentences.append(candidate)
                                        
                        else:
                            print("issue")
                            print(refer_sentences_file)
                            print(file)    
                        
                       
                        parser1 = etree.XMLParser(recover=True)
                      
                        #print("Test:"+xmlCitText)
                        try:
                            xmlEntryCit = "<root>"+xmlCitText.replace("sid = ","sid =\"0\"")+"</root>"
                          
                            xmlCitTree = ET.fromstring(xmlEntryCit.encode('UTF-8', 'ignore'),parser1)
                        except:
                            print("Except:"+xmlEntryCit)
                            print("\n"+str(counter))
                            raise
                       
                        
                        
                        citEntries = parseSentXMLtree(xmlCitTree, citOffsets)
                                                
                        for citEntry in citEntries:
                            if(citEntry[0]!=None):
                                for negSent in negSentences:
                                    negSentEnt = negSent.split("\t")  
                                    pair_off = citEntry[0] + "\t" + negSentEnt[0] + "\t" + str(citEntry[1]) + "\t" + str(citEntry[2]) + "\t" + str(negSentEnt[1]) + "\t" + str(negSentEnt[2]) + "\t" + name_C
                                    pair = citEntry[0] + "\t" + negSentEnt[0] 
                                    paircount = paircount+1
                                    wf1.write(pair_off + "\n")
                                    wf2.write(pair + "\n")
                            else:
                                print(citEntry)
                        
                        all_cit = ""
                        all_sid = []
                        all_ssid = []
                        if len(citEntries)>1:
                            print(citEntries)
                            for citEntry in citEntries:
                                all_cit+=citEntry[0]+" "
                                all_sid.append(citEntry[1])
                                all_ssid.append(citEntry[1])
                            all_cit = all_cit.strip()
                            for negSent in negSentences:
                                negSentEnt = negSent.split("\t")
                                pair_off = all_cit + "\t" + negSentEnt[0] + "\t" + str(all_sid) + "\t" + str(all_ssid) + "\t" + str(negSentEnt[1]) + "\t" + str(negSentEnt[2]) + "\t" + name_C
                                pair = all_cit + "\t" + negSentEnt[0] 
                                wf3.write(pair_off + "\n")
                                wf4.write(pair + "\n")
                            citEntries.clear()
                        negSentences.clear()

print(paircount)         
print(counter)

In [ ]:
# Process test set
import csv
initDir = "CL-SciSumm/scisumm-corpus-master/data/Test-Set-2018/"
testExt = ".csv"
refExt = ""
year="2018"
writeDir1 = "CL-SciSumm/scisumm-corpus-master/data/Testing_pairs_offsets-"+year+"/"
writeDir2 = "CL-SciSumm/scisumm-corpus-master/data/Testing_pairs-"+year+"/"

if not os.path.exists(writeDir1):
    os.makedirs(writeDir1)
if not os.path.exists(writeDir2):
    os.makedirs(writeDir2)

    

#parse test csv
for root, dirs, files in os.walk(initDir):
    for file in files:    
         if testExt in file:
            with open (os.path.join(root, file), "r") as testFile:          
                ref_name = file.split(".")[0]
                ref_file = "CL-SciSumm/scisumm-corpus-master/data/Test-Set-"+year+"/"+ref_name+"/Reference_XML/"+ref_name+".xml"
                sentences = []
                #parse reference xml
                with open(ref_file, encoding='utf-8', errors='ignore') as xml2str:
                    data=xml2str.read()
                    parser = ET.XMLParser()
                    xmlTree = ET.fromstring(data,parser)

                    for sent in xmlTree.iter("S"):
                        sid = sent.attrib['sid']
                        if sid == "0" or sid =="":
                            ssid = "0"
                        else:
                            ssid = sent.attrib['ssid']
                        senText = sent.text
                        sentVec = [sid,ssid,senText]
                        
                        sentences.append(sentVec)
                
                wf1 = open(writeDir1+ref_name+"_pairs_off.tsv", "w") 
                wf2 = open(writeDir2+ref_name+"_pairs.tsv", "w") 
                #parse csv test file
                csv_reader = csv.reader(testFile, delimiter=',')
                line_count = 0
                for row in csv_reader:
                    if line_count == 0:
                        print(f'Column names are {", ".join(row)}')
                        line_count += 1
                    else:
                        print(len(row))
                        cit_num = row[0]
                        ref_art = row[1]
                        cit_art = row[2]
                        cit_off = row[5]
                        cit_text = row[6]
                        cit_text_c = row[7]
                        for sentV in sentences:
                            pair = cit_text_c + "\t" + sentV[2]
                            pair_off = cit_text_c + "\t" + sentV[2] + "\t" + cit_art + "\t" + cit_off + "\t" + sentV[0] + "\t" + sentV[1] + "\t" + cit_art + "\t" + cit_num
                            print(pair)
                            wf2.write(pair + "\n")
                            wf1.write(pair + "")
                    
                   
                
                

In [ ]:
#Aug 2019
#modifying data to use with fine-tune approach
input_negative_pairs = "NN_Training_pairs_offsets_full_corr_neg-2019/"
output_dir= "20019-FT"
#outputDir2= "/home/chryssa/Work/CL-SciSumm/scisumm-corpus-master/data/toMove/2018/train/fullCitance/sent2vec/references/"

if not os.path.exists(output_dir):
    os.makedirs(output_dir)
    
trainLines=[]
counter=0


wf = open(os.path.join(output_dir,"2019_cl_corpus_negative_pairs" "w"))
for root, dirs, files in os.walk(input_negative_pairs):
    for file in files:    
        with open (os.path.join(root, file), "r") as annFile:
            for line in annFile:
                fields = line.split("\t")
                wf.write(fields[0])
                wf.write("\n")
                wf.write(fields[1])
                wf.write("\n")
                wf.write("\n")
wf.close()